# Writing an application: Identifying a coast line.

NOTE: This example is highly contrived and for pedagogical use only, please do not identify coastlines like this in practice, consult with a domain expert :)


## Introduction
Satellite imagery has been obtained of "Guernsey", the channel island, and surrounding smaller islands.

The task is to develop an image processing pipeline using the Numba compiler to speed up the detection of the edges of the islands and compute an estimate for the length of the coast line. A lot of the work needed to do this has been done already and the idea here is to explore writing functions and piecing them together to build up a small application.

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt

#### LICENSING NOTE:

**Attribution:** The image `Guernsey.png` has the following attribution:
["Guernsey by Sentinel-2.jpg"](https://en.wikipedia.org/wiki/File:Guernsey_by_Sentinel-2.jpg) by user `"Satview"`, original author `"Copernicus Sentinel-2, ESA"`, used under [CC BY-SA 3.0 IGO](https://creativecommons.org/licenses/by-sa/3.0/igo/deed.en) / Made smaller and changed format from original.

In [ ]:
fname = "Guernsey.png"
imdata = imageio.imread(fname)
plt.imshow(imdata)

<h3><span style="color:blue"> Task 1: Write a blurring function</span></h3>

Write a blurring function, perhaps average of left, right, above and below cells to start with?
More advanced blurs are of course available e.g. [see this in the Numba examples](https://github.com/numba/numba/blob/master/examples/gaussian-blur/gaussian-blur.py#L22-L28) if you want to upgrade later. Here's a stencil to help:

```
            [0, +1]
               |
               |
 [-1, 0] -- [0, 0] -- [0, +1]
               |
               |
            [0, -1]
```

In [ ]:
from numba import stencil

@stencil
def my_blur(img):
     # return uint8, this is an image!
    return np.uint8(# You write this!)

The next cell contains a few functions to get started, there's plenty of scope to optimise these!

In [ ]:
from numba import njit, types, objmode, stencil, vectorize
from numba.typed import List

@njit
def import_image(name):
    """
    Imports an image with the given name.
    Returns a N*M*3 uint8 array.
    """
    # use "objmode" context manager, jump back into the python interpreter for the context
    # managed block: http://numba.pydata.org/numba-doc/latest/user/withobjmode.html
    with objmode(ret="uint8[:,:,:]"):
        ret = imageio.imread(name)
    return ret.copy()

@njit
def channel_select(img, channel):
    """
    Returns the channel "channel" from the image "img"
    """
    return img[:, :, channel]

@njit
def channel_combine(ch0, ch1, ch2):
    """
    Combines three channels into a single image representation.
    """
    shape = ch0.shape + (3,)
    out = np.empty(shape, ch0.dtype)
    out[:, :, 0] = ch0
    out[:, :, 1] = ch1
    out[:, :, 2] = ch2
    return out

@njit
def compress(img, pc):
    """
    Compress the image "img" via SVD, "pc" a percentage as a decimal in the range 0 <= pc <=1
    to express the amount of components to keep.
    """
    u, sv, vt = np.linalg.svd(img.astype(np.float64))
    s = np.zeros((u.shape[1], vt.shape[0]))
    n = int(pc * len(sv))
    for i in range(n): # loops are cheap!
        s[i, i] = sv[i]
    return np.dot(np.dot(u, s), vt).astype(np.uint8)

@njit
def sobel(img):
    """
    A Sobel filter, good for detecting edges!
    Implementation based on https://en.wikipedia.org/wiki/Sobel_operator#Pseudocode_implementation
    """
    Gx = np.array([-1, 0, 1, -2, 0, 2, -1, 0, 1]).reshape((3, 3))
    Gy = np.array([-1, -2, -1, 0, 0, 0, 1, 2, 1]).reshape((3, 3))
    r, c = img.shape
    threshold = 70
    mag = np.zeros_like(img)
    for i in range(0, r - 2):
        for j in range(0, c - 2):
            box = img[i:i + 3, j:j + 3]
            S1 = np.sum(Gx * box)
            S2 = np.sum(Gy * box)
            sz = np.sqrt(S1 ** 2 + S2 ** 2)
            if sz < threshold:
                mag[i + 1, j + 1] = 255
    return mag


@njit
def to_grey_scale(R, G, B):
    """
    Combines R,G,B components to grey-scale.
    Luma model: https://en.wikipedia.org/wiki/Luma_(video)
    """
    return (0.2126 * R + 0.7152 * G + 0.0722 * B).astype(np.uint8)

<h3><span style="color:blue"> Task 2: Build an image processing pipeline</span></h3>

Using the functions above, and including your own (you might want to write a cropping function!?!), build a processing pipeline.

In [ ]:
# Perhaps write a cropping function here once you have a pipeline working?!

In [ ]:
# DIY pipeline
@njit
def pipeline(fname):
    """
    Processes the image with name specified by "fname"
    """
    # import the image with name, fname
    image = import_image(fname)
    # loop over channels doing any operations you like
    l = []
    for ch in range(3):
        im = channel_select(image, ch)
        # perhaps start with your blur? then try some other things?
        # - blurring averages out the features
        # - compression can be used to find the stronger features
        # - the sobel filter is good for edge detection

        im = # Call your blur function !
        
        # store
        l.append(im)
        
    # recombine to grey-scale
    combined = to_grey_scale(l[0], l[1], l[2])
    return crop(combined, 0.05, 0.05, 0.05, 0.05)

# run the pipeline and take a look!
out = pipeline("Guernsey.png")
plt.imshow(out, cmap='gray', vmin=0, vmax=255)

<h3><span style="color:blue"> Task 3: Count the pixels!</span></h3>

Write a function to count the number of values above a threshold value, use an explicit parallel `prange` loop.


In [ ]:
from numba import prange

@njit(parallel=True)
def coast_line_length(processed_image, threshold=0):
    r, c = processed_image.shape
    total = 0
    # You write this!
    return total

coast_line_length(out, 100)